<a href="https://colab.research.google.com/github/Feryalchemist/Point_Gamma-Radiation_Dose/blob/main/Point_Gamma-Radiation_Dose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**$\gamma$ External Radiation Dose**

This article will explain about a method to calculate the radiation dose that exposed through a source outside human body.  This article is focused on the gamma radiation since it has discrete energy and is not affected by coulomb force (don't worry about Bragg peak, moreover Bethe-Bloch formula). Although so, this calculation still an over simplified approach. So, to calculate accurately the particle simulation is more prefered.

Hereby, I'll show the script.

##**1. Packages**

**First**, we need to import several packages such as numpy and pandas, also, the table of photon cross section for the shielding. The table can be obtained using [this URL](https://physics.nist.gov/PhysRefData/Xcom/html/xcom1.html) provided by NIST XCOM. **The table is in .xlsx format with each sheet correspond to different material cross section table.**

Here, I provide the formatted table for various materials that I got from NIST table as an example. The materials are; H$_2$O, Al, Pb, Air (0.8 N$_2$, 0.2 O$_2$ w/o), Acrylic/Lucite/Plexiglas (C$_5$O$_2$H$_8$), NaI(Tl), ZnS(Ag), Steel (0.105 Cr, 0.012 C, Fe w/o) and element consist in Concrete. All the units are in **cm$^2$/g** and Energy in **MeV**.

In [ ]:
import math
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile

In [ ]:
from google.colab import files
uploaded = files.upload()
tab_name = list(uploaded.items())[0][-1]
mat_list = pd.ExcelFile(tab_name).sheet_names

print("\n Upload dat.BuildUp_Factor table data = \n")
uploaded = files.upload()

print("\n Upload dat.Emission_Data gamma emission table data = \n")
uploaded = files.upload()

print("\n\n------------------------------\nYour Available Material is = \n")
print(*mat_list, sep=' ') 

Saving CS_Table.xlsx to CS_Table.xlsx


------------------------------
Your Available Material is = 

Water Lucite NaI(Tl) ZnS(Ag) Steel Air Al Pb H C O Na Mg Si K Ca Fe N Zn S Ag Cr I Tl Be


##**2. Function**

Before we begin the calculation, we need to define the function to help our problem. In this second step, I will define several important fuction.

###**Linear Interpolation**
The interpolation will help us to find the value that unwritten in the table. In simple way, we can utilize linear interpolation to calculate value between two value written in the table.

$y = f(x)$

$y = \dfrac{(x-x_1)(y_2-y_1)}{(x_2-x_1)} + y_1$


In [ ]:
def Interpolation(x1,y1,x2,y2,x):
  y=(x-x1)*(y2-y1)/(x2-x1)+y1
  return y

###**Gamma Property**

There are some important parameter of radionuclide source, in which determine the radiation dose. Some radionuclide may emits multiple energy with varied emmision probability. This function will extract multiple gamma energy of a determined radionuclide in the **Emission_Data** table file. The data can be obtained from [here](https://www-pub.iaea.org/MTCD/Publications/PDF/Pub1287_Vol1_web.pdf).

where the properties includes:


1.   $f_i$ = energy probability
2.   $E_i$ = emitted energy in [keV]


You can add more if needed, just follow the data pattern.


In [ ]:
Gammadf = pd.read_csv('./dat.Emission_Data',header=[0],delimiter='\t')

def Gamma_property(Nuclide,Gammadf):
  E=[]; f=[]
  stdev_E=[]; stdev_f=[]
  for i in range(0,len(Gammadf)):
    if Gammadf['Nuclide'].iloc[i]==Nuclide:
	    E.append(Gammadf['E'].iloc[i])
	    stdev_E.append(Gammadf['stdev-E'].iloc[i])
	    f.append(Gammadf['f'].iloc[i])
	    stdev_f.append(Gammadf['stdev-f'].iloc[i])
     #Energy in keV
  return (np.array(E),np.array(f),np.array(stdev_E),np.array(stdev_f))

###**Attenuation Coefficient**

Attenuation coefficient is a coefficient that determine the reduction of radiation intensity when it pass through certain thickness of medium. Attenuation coefficient is based on the radiation energy. The intensity of radiation with initial intensity $I_0$ that pass through n-type of medium with n-different thickness ($\Delta x$) is:

###$I = I_0 e^{-\sum^n_{i=1}{\left(\mu_i\Delta x_i\right)}}$

The unit we used from XCOM is already normalized to mass, thus, the equation for attenuation coefficient for a compound medium with n-component will be:

###$\mu(E_e)_{_{[cm^{-1}]}}=\sum^n_{i=1}{\sigma_{i,tot}(E_e)_{_{[cm^{2}/g]}}\rho_{tot}}_{_{[g/cm^{3}]}}\small{\dfrac{M_i}{\sum^n_{i=1}{M_i}}}$

where $\rho_{tot}$ is the compound medium density, $M_{i}$ is the component-i atomic mass in unit of $_{g/mole}$. The value for each energy should be calculated separately.







In [ ]:
def Attenuation_coeff(E,M,Density,tab_name,CS = 'Tot. w/ Coherent'):
  Energy,Fraction = E
  Medium,Ratio    = M
  mu=np.zeros((len(Energy),len(Medium)))
  for m in range(0,len(Medium)):
    Data = pd.read_excel(tab_name,sheet_name = Medium[m],header=[0])
    print(' ----------------------------------------------------- ')
    for n in range(0,len(Energy)):
      i=0
      while Energy[n]-Data['Photon Energy'].iloc[i]>0:
        i+=1
      mu[n,m]=(Interpolation(Data['Photon Energy'].iloc[i-1],Data[CS].iloc[i-1],
                             Data['Photon Energy'].iloc[i],Data[CS].iloc[i],Energy[n]))
      print('mu = '+'{:.3e}'.format(mu[n,m])+
            '\tcoeff E = '+'{:.3e}'.format(Energy[n])+' // '+'{:.3e}'.format(Fraction[n]*100)+'%'+
            '\tMedium  = '+Medium[m]+' // '+'{:.3e}'.format(Ratio[m]*100)+'%')
  M_grid,E_grid=np.meshgrid(Ratio*Density,Fraction)
  total_attenuation=np.sum(M_grid*mu,axis=1)
  
  return (total_attenuation)

###**Buildup factor**

Build up factor is factor that determine the multiplication of radiation dose because of refocused ray after scattered by a shielding material. The value determined by the attenuation factor, thus it also related to radiation energy.

Build Up Factor obtained from **'Appendix E'** of this [reference](https://doi.org/10.1201/9781003009849) and evaluated with equation of:
###**$B=1+a \left(\dfrac{\mu}{\rho_{tot}}\right) r e^{b \left(\dfrac{\mu}{\rho_{tot}}\right) r}$**

where :

*   $a$ and $b$ = energy and cosine related  factor
*   $\mu$       = attenuation coefficient
*   $r$         = shielding thickness

Program will search $a$ and $b$ by checking the table energy value (in MeV).

```
Available shielding material = Concrete Air Water Fe Pb
```


In [ ]:
Appendix_E = pd.read_csv('./dat.BuildUp_Factor',header=[0],delimiter='\t')

def buildup(Material,E,mu,r,rho,Appendix_E):
  i=0
  while E-Appendix_E['E'].iloc[i]>0:
    i+=1
  a=Interpolation(Appendix_E['E'].iloc[i-1],Appendix_E[Material+'-a'].iloc[i-1],
                  Appendix_E['E'].iloc[i],Appendix_E[Material+'-a'].iloc[i],E)
  b=Interpolation(Appendix_E['E'].iloc[i-1],Appendix_E[Material+'-b'].iloc[i-1],
                  Appendix_E['E'].iloc[i],Appendix_E[Material+'-b'].iloc[i],E)
  B=1+a*(mu/rho)*r*math.exp(b*(mu/rho)*r)
  print('a = '+'{:.5e}'.format(a))
  print('b = '+'{:.5e}'.format(b))
  print('B = '+'{:.5e}'.format(B))
  return B

##**3. Calculation**

Now, lets move to the calculation. The gamma source strength with distance 1 m from isotropic point source is calculated by this equation :

###$\Gamma \left[\frac{J/kg.m^2}{MBq.h}\right](E)=\frac{f(\frac{photons}{d})E(\frac{MeV}{photon})1.6e-13(\frac{J}{MeV})1e6(\frac{d/s}{MBq})3.6e3(\frac{s}{h})\mu (\frac{1}{cm})}{4\pi \rho(\frac{g}{cm^3})}$

$J/kg$ can be replaced by gray, since they have same meaning which is "The amount of radiation energy absorbed per kilogram of medium". Therefore, **attenuation coefficient in this equation is for the medium**.


If the nuclide emits multiple radiation energy, then the calculation for each energy must be done separately and summed afterward as such:

$\sum_{e=1}^{n}{\Gamma_{tot}} = \Gamma_1+\Gamma_2+...+\Gamma_n$


###**Absorbed Dose Calculation ($\dot{D}$)**

The calculation breaks into 4 part which are:

1.   User Input   
2.   Get the Attenuation coefficient and Radiation Property
3.   Calculate Dose Rate after travel
4.   Calculate Dose Rate shielded

In this example we wanted to calculate the absorbed radiation dose emitted from Mo$^{99}$ that passed through water pool and shielded by Aluminum and Concrete (The composition obtained [here](https://geant4-userdoc.web.cern.ch/UsersGuides/ForApplicationDeveloper/html/Appendix/materialNames.html) provided by GEANT4 documentation). The target stand 1 cm above the pool.

Here is the complete illustration of our problem:

(*)<------ 30 cm Water ------>||[<------ 1 cm Al ------->]||[<------- 5 cm Concrete ----->]||<------ 1 cm Water ------>     (┛ಠ_ಠ)┛彡┻━┻


####**--Problem and User Input**

Here user can input the radiation source, Travelling Medium and Shielding. The medium is the first material in contact with the radiation, which in our case is Water. The rests is considered as shielding (even it it is the same material as medium). In this example case, User should input Water as medium, and Al, Concrete, Water as Shielding.

**$_{Separate\ the\ multiple\ input\ by\ 'space'}$**

**$_{Fraction\ means\ atomic\ fraction}$**

**$_{Fraction\ not\ equal\ to\ 1\ will\ be\ normalized'}$**

Element in Concrete:


H C O Na Mg Al Si K Ca Fe


0.01 0.001 0.529107 0.016 0.002 0.033872 0.337021 0.013 0.044 0.014

In [ ]:
# Input Source Nuclide
Source   = input('Radiation Source is [format = Cs137] = ')

# Input Medium with its Composition
Medium   = input('\n|--------Radiation Medium consists of = ')
Med_fr   = input('\n|-----------------------with Fraction = ')
Med_dx   = float(input('\n|---with distance to shield or target = '))
Med_den  = float(input('\n|------------------------with density = '))
Med_fr   = np.array([float(x) for x in Med_fr.split(' ')])

# Input The Shieldings and their Composition
Shield = []

finish = False
print("\n ---Type 'Stop' to Finish input & 'Continue' to Continue--- ")
while finish==False:
  Test = input('\n ---> ')
  if Test == 'Continue':
    S = {}
    S['Comp']      = input('\n Shielding '+str(len(Shield)+1)+' consists of = ').split(' ')
    S['Frac']      = input('\n |---------with Fraction = ').split(' ')
    S['thickness'] = float(input('\n |--------with Thickness = '))
    S['density']   = float(input('\n |----------with Density = '))
    S['Frac']      = np.array([float(x) for x in S['Frac']])/np.sum([float(x) for x in S['Frac']])
    Shield+= [S]
  else:
    finish=True

Radiation Source is [format = Cs137] = Mo99

|--------Radiation Medium consists of = Water

|-----------------------with Fraction = 1

|---with distance to shield or target = 30

|------------------------with density = 0.997

 ---Type 'Stop' to Finish input & 'Continue' to Continue--- 

 ---> Continue

 Shielding 1 consists of = Al

 |---------with Fraction = 1

 |--------with Thickness = 1

 |----------with Density = 2.7

 ---> Continue

 Shielding 2 consists of = H C O Na Mg Al Si K Ca Fe

 |---------with Fraction = 0.01 0.001 0.529107 0.016 0.002 0.033872 0.337021 0.013 0.044 0.014

 |--------with Thickness = 5

 |----------with Density = 1.8

 ---> Continue

 Shielding 3 consists of = Water

 |---------with Fraction = 1

 |--------with Thickness = 1

 |----------with Density = 0.997

 ---> Stop


#### **--Read Table of Attenuation, BuildUP, and Gamma Emission**

Here we utilize all the table reader function from section 2.

In [ ]:
# Get the Gamma Emission of Source
E,f,stdev_E,stdev_f = Gamma_property(Source,Gammadf)

# Get the BuildUp Fraction
build_up   = np.zeros(len(E))
mu_med     = Attenuation_coeff([E/1000,f],[[Medium],Med_fr],Med_den,tab_name)
for i in range(0,len(E)):
  print('\nBuildUp Equation Coefficient for energy','{:.3e}'.format(E[i]),'keV')
  build_up[i]= buildup(Medium,E[i]/1000,mu_med[i],Med_dx,Med_den,Appendix_E)

# Get the Attenuation Coefficient for each Shielding
print('\nAttenuation Coefficient for Each Compound and Energy = ')
mu_shield = []
for shield in Shield:
  mu_shield += [Attenuation_coeff([E/1000,f],[shield['Comp'],shield['Frac']],shield['density'],tab_name)]

 ----------------------------------------------------- 
mu = 2.659e-01	coeff E = 4.058e-02 // 1.022e+00%	Medium  = Water // 1.000e+02%
mu = 1.543e-01	coeff E = 1.405e-01 // 8.960e+01%	Medium  = Water // 1.000e+02%
mu = 1.421e-01	coeff E = 1.811e-01 // 6.010e+00%	Medium  = Water // 1.000e+02%
mu = 1.103e-01	coeff E = 3.664e-01 // 1.194e+00%	Medium  = Water // 1.000e+02%
mu = 8.196e-02	coeff E = 7.395e-01 // 1.212e+01%	Medium  = Water // 1.000e+02%
mu = 7.986e-02	coeff E = 7.779e-01 // 4.280e+00%	Medium  = Water // 1.000e+02%

BuildUp Equation Coefficient for energy 4.058e+01 keV
a = 2.68381e+00
b = 2.59247e-02
B = 2.72302e+01

BuildUp Equation Coefficient for energy 1.405e+02 keV
a = 5.11343e+00
b = 1.24051e-01
B = 4.28486e+01

BuildUp Equation Coefficient for energy 1.811e+02 keV
a = 4.41359e+00
b = 1.20647e-01
B = 3.23270e+01

BuildUp Equation Coefficient for energy 3.664e+02 keV
a = 2.84125e+00
b = 8.53726e-02
B = 1.34225e+01

BuildUp Equation Coefficient for energy 7.395e+02 keV
a =


####**-- Calculate Dose to the shielding**

In this part we can calculate dose rate using the following equation:

$\dot{D}[Gray/h]=\Gamma \dfrac{A[MBq] \times B(E)}{d^2[cm^2]}$

where $d$ is the distance from source to shielding or to target (if there is no shielding).

In [ ]:
A = float(input('Input source Activity [MBq] = '))

G = E*f*1.6e-13*1e6*3.6e3*mu_med/(4*math.pi*Med_den)
D = G*A*build_up/(Med_dx**2)

print('\n Absorbed Dose in Al face is = ')
for e,d in zip(E,D):
  print('{:.5e} MeV   ----   {:.5e} mGy/h'.format(e/1000,d*1e3))

Input source Activity [MBq] = 1

 Absorbed Dose in Al face is = 
4.05832e-02 MeV   ----   1.52937e-04 mGy/h
1.40511e-01 MeV   ----   4.24019e-02 mGy/h
1.81094e-01 MeV   ----   2.54638e-03 mGy/h
3.66421e-01 MeV   ----   3.29877e-04 mGy/h
7.39500e-01 MeV   ----   2.27779e-03 mGy/h
7.77921e-01 MeV   ----   7.77661e-04 mGy/h


####**-- Calculate Dose to the target**

This is the final result. The dose for each energy level are obtained using the exponential relation mentioned in Attenuation function section. The final result is the summation of dose in all energy level. 

$\dot{D} = \sum^n_{i=1}{\dot{D}(E_i)e^{-\sum^{m}_{j=1}{\mu_j(E_i)\Delta x_j}}}$

where n is number of energy level and m is number of shielding layer.

You can input the weighting factor of tissue to calculate equivalent dose in unit of Sv/h.

In [ ]:
for n,mu in enumerate(mu_shield):
  D *= np.exp(-mu*Shield[n]['thickness'])

print('\n Absorbed Dose in target is = ')
for e,d in zip(E,D):
  print('{:.5e} MeV   ----   {:.5e} mGy/h'.format(e/1000,d*1e3))
print('\n ---------------------------- \n',
      'total = {:.5e} mGy/h'.format(np.sum(D*1e3)))

wr  = float(input('\nInput the weighting Factor = '))
D_E = 1*wr*np.sum(D)

print('\n ---------------------------- \n',
      'equivalent dose = {:.5e} mSv/h'.format(D_E))



 Absorbed Dose in target is = 
4.05832e-02 MeV   ----   3.21094e-13 mGy/h
1.40511e-01 MeV   ----   1.49041e-04 mGy/h
1.81094e-01 MeV   ----   1.62297e-05 mGy/h
3.66421e-01 MeV   ----   7.16213e-06 mGy/h
7.39500e-01 MeV   ----   1.34291e-04 mGy/h
7.77921e-01 MeV   ----   4.93193e-05 mGy/h

 ---------------------------- 
 total = 3.56043e-04 mGy/h

Input the weighting Factor = 0.12

 ---------------------------- 
 equivalent dose = 4.27252e-08 mSv/h
